In [1]:
import pandas as pd

In [2]:
adv_stats = pd.read_csv("adv_stats.csv")
hist_odds = pd.read_csv("hist_odds.csv")
mom_stats = pd.read_csv("momentum.csv")
today_lines = pd.read_csv("today_lines.csv")
team_id = {'Washington' : 1610612764, 'GoldenState' : 1610612744, 'Minnesota' : 1610612750, 'Houston' : 1610612745, 'Milwaukee' : 1610612749, 'Chicago' : 1610612741, 'Philadelphia': 1610612755, 'Memphis' : 1610612763, 'Brooklyn' : 1610612751, 
           'OklahomaCity' : 1610612760, 'Sacramento' : 1610612758, 'Charlotte' : 1610612766, 'Indiana' : 1610612754, 'Toronto': 1610612761, 'SanAntonio' : 1610612759, 'Orlando' : 1610612753, 'Utah' : 1610612762, 'NewOrleans' : 1610612740, 'LALakers': 1610612747, 
           'Portland' : 1610612757, 'Dallas' : 1610612742, 'Atlanta' : 1610612737, 'Boston' : 1610612738, 'Cleveland' : 1610612739, 'Detroit': 1610612765, 'Miami' : 1610612748, 'LAClippers' : 1610612746, 'Denver' : 1610612743, 'Phoenix' : 1610612756, 'NewYork' : 1610612752}
hist_odds = hist_odds.replace(team_id)
hist_odds['win'] = [1 if x > y else 0 for x,y in zip(hist_odds['Final_H'], hist_odds['Final_V'])]
hist_odds['fav'] = [1 if x > y else 0 for x,y in zip(hist_odds['ML_H'], hist_odds['ML_V'])]
home_team = hist_odds[["VH_y", "Team_H", "Final_H"]]
away_team = hist_odds[['VH_x', "Team_V", "Final_V"]]

home_team = pd.merge(left = home_team, right = adv_stats, left_on = "Team_H", right_on = "TEAM_ID", how = "left")
away_team = pd.merge(left = away_team, right = adv_stats, left_on = "Team_V", right_on = "TEAM_ID", how = "left")
model_stats = pd.merge(left = home_team, right = away_team, left_index = True, right_index = True, how = "outer")
model_stats['win'] = hist_odds['win']
model_stats['fav'] = hist_odds['fav']
model_stats.to_csv('model_stats.csv')

In [3]:
to_keep = ['W_PCT_x', 'W_PCT_y', 'VH_y', 'E_OFF_RATING_x', 'E_OFF_RATING_y', 'E_DEF_RATING_x', 'E_DEF_RATING_y', 'E_NET_RATING_x', 'E_PACE_x', 'E_AST_RATIO_x', 'E_OREB_PCT_x' , 'E_DREB_PCT_x',
           'E_REB_PCT_x', 'E_TM_TOV_PCT_x', 'E_NET_RATING_y' , 'E_PACE_y', 'E_AST_RATIO_y', 'E_OREB_PCT_y' ,'E_DREB_PCT_y', 'E_REB_PCT_y', 'E_TM_TOV_PCT_y', 'MIN_x', 'MIN_y', 'fav']
model_stats_x = model_stats[to_keep]
col_names = ['W_PCT_HOME', 'W_PCT_AWAY', 'home_team', 'E_OFF_RATING_HOME', 'E_OFF_RATING_AWAY', 'E_DEF_RATING_HOME', 'E_DEF_RATING_AWAY', 'E_NET_RATING_HOME', 'E_PACE_HOME', 'E_AST_RATIO_HOME', 'E_OREB_PCT_HOME' , 'E_DREB_PCT_HOME',
           'E_REB_PCT_HOME', 'E_TM_TOV_PCT_HOME', 'E_NET_RATING_AWAY' , 'E_PACE_AWAY', 'E_AST_RATIO_AWAY', 'E_OREB_PCT_AWAY' ,'E_DREB_PCT_AWAY', 'E_REB_PCT_AWAY', 'E_TM_TOV_PCT_AWAY', 'MIN_HOME', 'MIN_AWAY', 'fav']
model_stats_x = model_stats[to_keep]

Logistic Regression

In [4]:
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

logit_model=sm.Logit(model_stats['win'],model_stats_x)
result=logit_model.fit()
print(result.summary())


X_train, X_test, y_train, y_test = train_test_split(model_stats_x, model_stats['win'], test_size=0.3, random_state=0)
logreg = LogisticRegression(max_iter = 1000)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

C:\Users\thyang\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:49: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


Optimization terminated successfully.
         Current function value: 0.608973
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                    win   No. Observations:                  533
Model:                          Logit   Df Residuals:                      509
Method:                           MLE   Df Model:                           23
Date:                Sat, 13 Mar 2021   Pseudo R-squ.:                  0.1184
Time:                        18:47:29   Log-Likelihood:                -324.58
converged:                       True   LL-Null:                       -368.16
                                        LLR p-value:                 2.121e-09
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
W_PCT_x            4.2734      2.384      1.793      0.073      -0.398       8.945
W_PCT_y          

In [10]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import classification_report, confusion_matrix
X_train, X_test, y_train, y_test = train_test_split(model_stats_x, model_stats['win'], test_size=0.3, random_state=100)
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 
  
grid.fit(X_train, y_train) 
print(grid.best_params_) 
print(grid.best_estimator_) 

grid_predictions = grid.predict(X_test) 
print(classification_report(y_test, grid_predictions)) 

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END .....................C=0.1, gamma=1, kernel=rbf; total time=   0.0s
[CV 2/5] END .....................C=0.1, gamma=1, kernel=rbf; total time=   0.0s
[CV 3/5] END .....................C=0.1, gamma=1, kernel=rbf; total time=   0.0s
[CV 4/5] END .....................C=0.1, gamma=1, kernel=rbf; total time=   0.0s
[CV 5/5] END .....................C=0.1, gamma=1, kernel=rbf; total time=   0.0s
[CV 1/5] END ...................C=0.1, gamma=0.1, kernel=rbf; total time=   0.0s
[CV 2/5] END ...................C=0.1, gamma=0.1, kernel=rbf; total time=   0.0s
[CV 3/5] END ...................C=0.1, gamma=0.1, kernel=rbf; total time=   0.0s
[CV 4/5] END ...................C=0.1, gamma=0.1, kernel=rbf; total time=   0.0s
[CV 5/5] END ...................C=0.1, gamma=0.1, kernel=rbf; total time=   0.0s
[CV 1/5] END ..................C=0.1, gamma=0.01, kernel=rbf; total time=   0.0s
[CV 2/5] END ..................C=0.1, gamma=0.0

[CV 3/5] END ....................C=1000, gamma=1, kernel=rbf; total time=   0.0s
[CV 4/5] END ....................C=1000, gamma=1, kernel=rbf; total time=   0.0s
[CV 5/5] END ....................C=1000, gamma=1, kernel=rbf; total time=   0.0s
[CV 1/5] END ..................C=1000, gamma=0.1, kernel=rbf; total time=   0.0s
[CV 2/5] END ..................C=1000, gamma=0.1, kernel=rbf; total time=   0.0s
[CV 3/5] END ..................C=1000, gamma=0.1, kernel=rbf; total time=   0.0s
[CV 4/5] END ..................C=1000, gamma=0.1, kernel=rbf; total time=   0.0s
[CV 5/5] END ..................C=1000, gamma=0.1, kernel=rbf; total time=   0.0s
[CV 1/5] END .................C=1000, gamma=0.01, kernel=rbf; total time=   0.0s
[CV 2/5] END .................C=1000, gamma=0.01, kernel=rbf; total time=   0.0s
[CV 3/5] END .................C=1000, gamma=0.01, kernel=rbf; total time=   0.0s
[CV 4/5] END .................C=1000, gamma=0.01, kernel=rbf; total time=   0.0s
[CV 5/5] END ...............